# Post Processing

**In this notebook:**

* Test DBSCAN for post processing

## Dependencies
Install, load, and initialize all required dependencies for this experiment.

### Install Dependencies

In [ ]:
#It should be possible to run the notebook independent of anything else. 
# If dependency cannot be installed via pip, either:
# - download & install it via %%bash
# - atleast mention those dependecies in this section

import sys
!{sys.executable} -m pip install -q -e ../../utils/

^C
ERROR: Operation cancelled by user
You should consider upgrading via the '/opt/jupyterhub/bin/python3 -m pip install --upgrade pip' command.


# System libraries

In [2]:
from __future__ import absolute_import, division, print_function
import logging, os, sys

# Enable logging
logging.basicConfig(format='[%(levelname)s] %(message)s', level=logging.INFO, stream=sys.stdout)

# Re-import packages if they change
%load_ext autoreload
%autoreload 2

# Recursion Depth
import sys
sys.setrecursionlimit(10000)

# Intialize tqdm to always use the notebook progress bar
import tqdm
tqdm.tqdm = tqdm.tqdm_notebook

# Third-party libraries
import comet_ml
import numpy as np
import pandas as pd
import nilearn.plotting as nip
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import collections
%matplotlib inline
plt.rcParams["figure.figsize"] = (12,6)
%config InlineBackend.figure_format='retina'  # adapt plots for retina displays
import git


# Project utils

import aneurysm_utils
from aneurysm_utils import evaluation, training


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
if "workspace" in os.getcwd():
    ROOT = "/workspace"
elif "/group/cake" in os.getcwd(): 
    ROOT = "/group/cake"

### Initialize Environment

In [4]:
env = aneurysm_utils.Environment(project="our-git-project", root_folder=ROOT)
env.cached_data["comet_key"] = "EGrR4luSis87yhHbs2rEaqAWs" 
env.print_info()

Environment Info:

Library Version: 0.1.0
Configured Project: our-git-project

Folder Structure: 
- Root folder: /group/cake
 - Project folder: /group/cake/our-git-project
 - Datasets folder: /data/training
 - Models folder: /group/cake/our-git-project/models
 - Experiments folder: /group/cake/our-git-project/experiments


## Load Data
Download, explore, and prepare all required data for the experiment in this section.

In [5]:
dataset_params = {
    "prediction": "labeled",
    "mri_data_selection": "", 
    "balance_data": False,
    "seed": 1,
    "resample_voxel_dim": (1,1,1)
}

preprocessing_params = {
    'min_max_normalize': None,
    'mean_std_normalize': False,
    'smooth_img': False, # can contain a number: smoothing factor
}


### Load Meta Data

In [6]:
from aneurysm_utils.data_collection import load_aneurysm_dataset

df = load_aneurysm_dataset(
    env,
    mri_data_selection=dataset_params["mri_data_selection"],
    random_state=dataset_params["seed"]
)
df.head()

,Aneurysm Geometry,Angiography Data,Vessel Geometry,Labeled Mask Index,Location,Age,Sex,Rupture Status,Age Bin,Aneurysm Count,Case,Path Orig,Path Mask,Path Vessel,Path Labeled Mask
0,A001.stl,A001_orig.nii.gz,A001_vessel.stl,1,Acom,48,m,1.0,"(40, 50]",1,A001,/data/training/A001_orig.nii.gz,/data/training/A001_masks.nii.gz,/data/training/A001_vessel.nii.gz,/data/training/A001_labeledMasks.nii.gz
1,A003.stl,A003_orig.nii.gz,A003_vessel.stl,1,Pcom,58,f,0.0,"(50, 60]",1,A003,/data/training/A003_orig.nii.gz,/data/training/A003_masks.nii.gz,/data/training/A003_vessel.nii.gz,/data/training/A003_labeledMasks.nii.gz
2,A005.stl,A005_orig.nii.gz,A005_vessel.stl,1,PICA,45,m,1.0,"(40, 50]",1,A005,/data/training/A005_orig.nii.gz,/data/training/A005_masks.nii.gz,/data/training/A005_vessel.nii.gz,/data/training/A005_labeledMasks.nii.gz
3,A006.stl,A006_orig.nii.gz,A006_vessel.stl,1,ACom,46,f,1.0,"(40, 50]",1,A006,/data/training/A006_orig.nii.gz,/data/training/A006_masks.nii.gz,/data/training/A006_vessel.nii.gz,/data/training/A006_labeledMasks.nii.gz
4,A008.stl,A008_orig.nii.gz,A008_vessel.stl,1,ACA,72,f,0.0,"(70, 80]",1,A008,/data/training/A008_orig.nii.gz,/data/training/A008_masks.nii.gz,/data/training/A008_vessel.nii.gz,/data/training/A008_labeledMasks.nii.gz


In [28]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(
        df,
        test_size=0.1,
        shuffle=True,
        random_state=1,
)

print(df_test["Case"])


115    A130_R
104      A118
106      A120
102      A115
119      A133
60       A073
99       A114
59       A072
53       A064
71       A084
63       A077
Name: Case, dtype: object


In [16]:
from aneurysm_utils.data_collection import split_mri_images

train_data, test_data, val_data, _ = split_mri_images(
    env, 
    df, 
    prediction=dataset_params["prediction"], 
    encode_labels=False,
    random_state=dataset_params["seed"],
    balance_data=dataset_params["balance_data"],
    resample_voxel_dim=dataset_params["resample_voxel_dim"]
)
mri_imgs_test, labels_test = test_data

train_data, test_data, val_data, _ = split_mri_images(
    env, 
    df, 
    prediction="mask", 
    encode_labels=False,
    random_state=dataset_params["seed"],
    balance_data=dataset_params["balance_data"],
    resample_voxel_dim=dataset_params["resample_voxel_dim"]
)
mri_imgs_test, invidual_labels_test = test_data

109
98
         Images
-----  --------
All         109
Train        87
Val          11
Test         11



  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

109
98
         Images
-----  --------
All         109
Train        87
Val          11
Test         11



  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [25]:
for image in labels_test:
    print(np.unique(image))
print(df_test["Aneurysm Count"])

[0. 1. 2. 3. 4.]
[0. 1.]
[0. 1.]
[0. 1. 2.]
[0. 1.]
[0. 1.]
[0. 1. 2. 3.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
115    4
104    1
106    1
102    2
119    1
60     1
99     3
59     1
53     1
71     1
63     1
Name: Aneurysm Count, dtype: int64


In [102]:
store_inviduals=invidual_labels_test
store_lables=labels_test

!free -m

from aneurysm_utils import data_collection,postprocessing 
mir_images, vessel_labels, patients = data_collection.load_mri_images(env,df,prediction="vessel",case_list=case_list,resample_voxel_dim=None)

case_list=["A001","A003","A005","A006","A008","A009","A010","A012","A014","A015","A016","A017_L_","A018","A019"]

mir_images, aneurysm_labels, patients = data_collection.load_mri_images(env,df,prediction="mask",case_list=case_list)
mir_images, invidiual_aneurysm_labels, patients = data_collection.load_mri_images(env,df,prediction="labeled",case_list=case_list)


In [59]:
new= np.load("../markus/predictions.npy")
converter = map(lambda x: x[0], new)
aneurysm_labels = list(converter)
len(aneurysm_labels)

162

for image in aneurysm_labels:
    print(np.unique(image))

In [67]:
def unify_alternativ(image, patch_size,most_common_shape):
    """
    image: numpy.array
          shape is (number_of_patches,h,w,d) , number_of_patches,heigth,width,depth
    patch_size: 64

    most_common_shape: tuple
                for example(139,139,120)

    Return

    image: np.array

          unified and unpatch image, shape (most_common_shape)


    """
    number_of_patches,heigth,width,depth = image.shape

    dim = np.array(most_common_shape)# size of the image
    n_patches = np.ceil(dim/patch_size).astype(int) # calculates the number of patches for each dim, to cover all voxel at least once in form e.g[3,3,2]
    rest  = n_patches * patch_size%dim ## calculates number of entries for each dimension which overlapp, means for example n_patches = 18 and 64 we have rest = 53
    
    h,w,d = most_common_shape

    ## initializing empty array for the unified image
    unified_image =  np.zeros([h,w,d])

    counter = 0 ## counter for patches
    print(counter)

    
    for i in range(n_patches[0]):

        if i == n_patches[0]-1: ## only the last cube is an overlapped cube
          start_x = i*patch_size-rest[0]
          stop_x= (i+1)* patch_size-rest[0]

        else:    
          start_x = i*patch_size
          stop_x = (i+1)* patch_size



        for j in range(n_patches[1]):
            if j == n_patches[1]-1: ## only the last cube is an overlapped cube
                start_y = j*patch_size-rest[1]
                stop_y= (j+1)* patch_size-rest[1]

            else:    
                start_y = j*patch_size
                stop_y = (j+1)* patch_size

            for k in range(n_patches[2]):
                if k == n_patches[2]-1: 
                    start_z = k*patch_size-rest[2]
                    stop_z = (k+1)* patch_size-rest[2]

                else:    
                    start_z = k*patch_size
                    stop_z = (k+1)* patch_size


              ##maximum between channel one or two, probably not necessary
              #max_class = np.maximum(image[n,0,:,:,:],image[n,1,:,:,:])

              ###includes overlapping case
              #unified_image[start_x:stop_x,start_y:stop_y,start_z:stop_z] = np.maximum(unified_image[start_x:stop_x,start_y:stop_y,start_z:stop_z],max_class)
                unified_image[start_x:stop_x,start_y:stop_y,start_z:stop_z] = np.maximum(unified_image[start_x:stop_x,start_y:stop_y,start_z:stop_z],image[counter,:,:,:])
                
                counter+=1## next patch
                
                if (counter==18):
                    break;
    
    return unified_image

In [68]:
def patch_unifier_alternativ(list_patches,size_test_set,most_common_shape,patch_size):
  """
  list_patches: list
                containing predictions from the evaluations, length should be number_of_patches x length_test_set
                each element should have the form (h,w,d)
  size_test_set: int

  most_common_shape: tuple
                most_common shape from the original input images before patchifying
  
  patch_size: int



  Return:

  unified_images: list
                contains the unpatched images 
                each element has the shape:(most_common_shape) 

  """
  
  dim = np.array(most_common_shape)
  n_patches = np.ceil(dim/patch_size).astype(int)# output is number of patches per dimension

  number_of_patches = np.prod(n_patches)# number of patches overall

  h,w,d = most_common_shape
  
  unified_images = []
  assert (len(list_patches)/(number_of_patches))== size_test_set
  

  ##output list: each element has the form (number_of_patches,n_classes,h,w,d)
  images = np.split(np.array(list_patches),size_test_set)  

  for n in range(size_test_set):
        
        unified_images.append(unify_alternativ(images[n],patch_size,most_common_shape))
  assert len(unified_images) == size_test_set
  return unified_images

In [94]:
aneurysm_labels_new=patch_unifier_alternativ(aneurysm_labels,9,(139,139,120),64)


0
0
0
0
0
0
0
0
0


In [70]:
len(aneurysm_labels_new)

9

In [95]:

postprocess_dict={ 
    "dbscan":True,
    "remove_border_candidates":True,
    "resample":True,
    "eps":2,
    "min_samples":30,
    #"evaluate_dbscan":True,
    "invidual_aneurysm_labels":invidual_labels_test,
    "size":90
                      
}

for imagebla in aneurysm_labels_new_test:
    print(np.unique(imagebla))

In [96]:
from aneurysm_utils import postprocessing
mri_imgs= postprocessing.postprocess(env,aneurysm_labels_new,postprocess_dict)


[INFO] Postprocessing: DBSCAN...
[INFO] Postprocessing: Removing noise...
[INFO] Postprocessing: Removing border candidates...
[0. 1.]
[INFO] Postprocessing: Resample to Size(256, 256, 220)


for image in invidual_labels_test:
    print(image.shape)
invidual_labels_test.pop(6)
invidual_labels_test.pop(8)
labels_test.pop(6)
labels_test.pop(8)
for image in invidual_labels_test:
    print(image.shape)

In [90]:
for image in mri_imgs:
    print(np.unique(image))

[0. 1.]
[0. 1. 2. 3. 4. 5.]
[0. 1. 2. 3. 4.]
[0. 1. 2.]
[0. 1. 2. 3. 4. 5. 6. 7. 8.]
[0. 1. 2. 3.]
[0. 1. 2. 3. 4. 5. 6.]
[0. 1. 2. 3. 4.]
[0. 1. 2. 3.]


In [130]:
for image in labels_test:
    print(np.unique(image))

[0. 1. 2. 3. 4.]
[0. 1.]
[0. 1.]
[0. 1. 2.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]


In [77]:
evaluation.draw_mask_3d(aneurysm_labels_new_test[1])

In [107]:
bounding_boxes =postprocessing.bounding_boxes(labels_test)

[0. 1. 2. 3. 4.]
[0. 1. 2. 3. 4.]
[0. 1. 2. 3. 4.]
[0. 1. 2. 3. 4.]
[0. 1.]
[0. 1.]
[0. 1. 2.]
[0. 1. 2.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]


std::vector<Eigen::Vector3d> with 8 elements.
Use numpy.asarray() to access data.


In [117]:
evaluation.draw_bounding_box(bounding_boxes[1]["candidates"],aneurysm_labels_new[1],aneurysm_array=labels_test[1])

In [133]:

print(np.where(labels_test[3]!=0))
evaluation.draw_mask_3d(labels_test[3])

AttributeError: 'tuple' object has no attribute 'T'

In [123]:
import pickle
# print(len(mri_imgs))

# with open("postprocessed_imgs.pkl", "wb") as fp:   
#     pickle.dump(mri_imgs, fp)

with open("postprocessed_imgs.pkl", "wb") as fp:   
    pickle.dump(mri_imgs, fp)
with open("labels_test.pkl","wb") as fp:
    pickle.dump(invidual_labels_test,fp)
with open("labels_test_invidual.pkl","wb") as fp:
    pickle.dump(labels_test,fp)
with open("neuralnet_output.pkl","wb") as fp:
    pickle.dump(aneurysm_labels_new,fp)

In [1]:
#evaluation.draw_bounding_box(bounding_boxes[9]["candidates"],aneurysm_array=mri_imgs[9],vessel_array=vessel_labels[9])

NameError: name 'evaluation' is not defined

In [154]:
from aneurysm_utils import evaluation

(256, 256, 220)


In [184]:
print(np.unique(labels_test[0]))

[0. 1. 2. 3. 4.]


In [57]:
#score_dict2=evaluation.calc_scores_task_2(aneurysm_labels_new,invidual_labels_test,mri_imgs,labels_test)
#score_dict1=evaluation.calc_scores_task_1(mri_imgs,labels_test,bounding_boxes)

685
676
36
33
24
20
12
4
685
0
36
0
24
0
12
0
685
0
36
0
24
0
12
0
685
0
36
0
24
0
12
0
685
0
36
0
24
0
12
0
685
0
36
0
24
0
12
0
685
0
36
0
24
0
12
0
478
0
478
0
478
0
181
0
106
0
181
0
106
0
181
0
106
0
165
0
165
0
34
0
22
0
7
0
34
0
22
0
7
0
34
0
22
0
7
0
34
0
22
0
7
0
34
0
22
0
7
0
34
0
22
0
7
0
42
0
42
0
42
0
42
0
42
0
36
0
36
0
36
0
36
0


In [58]:
print(score_dict1)

{'coverage_score': {'all_data': array([[1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]]), 'average': 0.1111111111111111}, 'bbox_fit_score': {'all_data': array([[0.60284707],
       [0.        ],
       [0.25570241],
       [0.28355416],
       [0.21227435],
       [0.        ],
       [1.        ],
       [0.63355907],
       [0.4880805 ]]), 'average': 0.3862241741634581}, 'f2_score': {'all_data': array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]]), 'average': 0.0}}


In [ ]:
final_score =evaluation.calc_total_segmentation_score(score_dict2)